In [44]:
import geopandas as gpd
import pandas as pd
from h3 import h3
from geomet import wkt
import json
from ast import literal_eval

---

### BUS STOPS

In [20]:
bus_stops = pd.read_parquet("../data/datalake/trusted/busstops")

In [21]:
h3_level = 10
 
def lat_lng_to_h3(row):
    return h3.geo_to_h3(
      float(row.latitude), float(row.longitude), h3_level)
 
bus_stops['h3'] = bus_stops.apply(lat_lng_to_h3, axis=1)

In [22]:
bus_stops.head()

,itinerary_id,seq,line_code,latitude,longitude,name,number,line_way,type,year,month,day,h3
0,626,3,311,-25.438212585461,-49.217415020767,"Rua Prof. Benedito Conceic?o, 770 - Cap?o da I...",130605,Araguaia,Placa em cano,2019,5,1,8aa831346947fff
1,671,8,378,-25.429452401161,-49.240590215868,"Av. Mal. Humberto de Alencar Castelo Branco, 1...",130412,Unimed,Novo mobiliario,2019,5,1,8aa83134425ffff
2,818,19,622,-25.493613012084,-49.331571663285,"Rua Professor Algacir Munhoz Mader, 3825 - Cid...",180299,Rondon,Chapeu chines,2019,5,1,8aa804d94a5ffff
3,7070,1,021,-25.435991809647,-49.307712311789,Terminal Campina do Siqueira - 021 - Interbair...,104506,Atendimento C.Siqueira - Cabral,Plataforma,2019,5,1,8aa83136d537fff
4,7010,8,022,-25.471695546865,-49.237493109333,Estac?o Tubo Salgado Filho,109065,Horario,Estac?o tubo,2019,5,1,8aa83136628ffff


### BAIRROS

In [37]:
shapefile = '../data/shape_files/DIVISA_DE_BAIRROS.shp'

bairros = gpd.read_file(shapefile,crs="epsg:29192")
#bairros.crs = "EPSG:4326"

In [38]:
bairros.crs

<Projected CRS: EPSG:29192>
Name: SAD69 / UTM zone 22S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - onshore and offshore northern and southern hemispheres between 54°W and 48°W. In remainder of South America - between 54°W and 48°W, southern hemisphere, onshore.
- bounds: (-54.0, -35.71, -47.99, 7.04)
Coordinate Operation:
- name: UTM zone 22S
- method: Transverse Mercator
Datum: South American Datum 1969
- Ellipsoid: GRS 1967 Modified
- Prime Meridian: Greenwich

In [39]:
bairros = bairros.to_crs("EPSG:4326") 

In [45]:
bairros = bairros[['CODIGO','NOME','CD_REGIONA','NM_REGIONA','geometry']]

#Apply function to create new column
bairros["geojson"] = bairros["geometry"].apply(lambda x: json.dumps(wkt.loads(str(x))))

In [46]:
bairros.head(1)

,CODIGO,NOME,CD_REGIONA,NM_REGIONA,geometry,geojson
0,9.0,ÁGUA VERDE,7.0,REGIONAL PORTÃO,"POLYGON ((-49.27115 -25.45303, -49.27086 -25.4...","{""type"": ""Polygon"", ""coordinates"": [[[-49.2711..."


In [47]:
bairros['h3'] = bairros["geojson"].apply(lambda x: list(h3.polyfill(literal_eval(x),10,geo_json_conformant=True)))

In [48]:
bairros.head()

,CODIGO,NOME,CD_REGIONA,NM_REGIONA,geometry,geojson,h3
0,9.0,ÁGUA VERDE,7.0,REGIONAL PORTÃO,"POLYGON ((-49.27115 -25.45303, -49.27086 -25.4...","{""type"": ""Polygon"", ""coordinates"": [[[-49.2711...","[8aa8313613b7fff, 8aa831361ad7fff, 8aa83136134..."
1,18.0,JARDIM SOCIAL,1.0,REGIONAL MATRIZ,"POLYGON ((-49.23 -25.41648, -49.23 -25.4165, -...","{""type"": ""Polygon"", ""coordinates"": [[[-49.2300...","[8aa831340cc7fff, 8aa83134002ffff, 8aa83134018..."
2,57.0,XAXIM,2.0,REGIONAL BOQUEIRÃO,"POLYGON ((-49.26498 -25.48821, -49.26412 -25.4...","{""type"": ""Polygon"", ""coordinates"": [[[-49.2649...","[8aa831acb4d7fff, 8aa831ac80dffff, 8aa831acac0..."
3,22.0,JARDIM DAS AMÉRICAS,3.0,REGIONAL CAJURU,"POLYGON ((-49.22781 -25.47203, -49.22881 -25.4...","{""type"": ""Polygon"", ""coordinates"": [[[-49.2278...","[8aa83136218ffff, 8aa83136221ffff, 8aa8313666a..."
4,26.0,GUAÍRA,7.0,REGIONAL PORTÃO,"POLYGON ((-49.28244 -25.47696, -49.28353 -25.4...","{""type"": ""Polygon"", ""coordinates"": [[[-49.2824...","[8aa83136579ffff, 8aa831361917fff, 8aa83136572..."


In [49]:
bairros = bairros[['CODIGO','NOME','CD_REGIONA','NM_REGIONA','h3']].rename(columns={"CODIGO": "code", "NOME":"name", "CD_REGIONA":"section_code",
                                                                                    "NM_REGIONA":"section_name","h3":"h3_index10"})
bairros.to_csv("bairros.csv", index=False)